# 1 - Importando as bibliotecas
  

In [ ]:
##INSTALAÇÃO DA BIBLIOTECA PYSPARK
!pip install pyspark
#INSTALAÇÃO DA BIBLIOTECA GOOGLE CLOUD STORAGE FILE SYSTEM
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importando o Gooogle Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



## 1.1 - Importando as funções e bibliotecas



In [ ]:
#IMPORTAR AS BIBLIOTECAS NECESSÁRIAS
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from google.cloud import storage
import os
import pandas as pd
from pyspark.sql.types import *
import numpy as np
import socket

In [ ]:
# Configurando as options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 100)

In [ ]:
#CONFIGURANDO DA CHAVE DE SEGURANCA (Enviada com o projeto)

serviceAccount = '/content/projeto-aquecimento-371716-63751ac5c63f.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

#2 - Enviando pro bucket da Google Cloud

In [ ]:
  #Configurações Google Cloud Storage

# Upando os arquivos originais para a bucket da GCP

client = storage.Client()

bucket = client.get_bucket('projetoaquecimente01')
blob = bucket.blob('INSS_ori/CAT_INSS_01_2022.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202201.csv')


bucket = client.get_bucket('projetoaquecimente01')
blob = bucket.blob('INSS_ori/CAT_INSS_04_2022.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202204.csv')


bucket = client.get_bucket('projetoaquecimente01')
blob = bucket.blob('INSS_ori/CAT_INSS_10_2022.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202210.csv')


# 3 -  Importando os arquivos do bucket da GCP

 Extraindo os dados da CAT cadastradas no sistema informatizado de "Comunicação de Acidentes do Trabalho" do INSS (CATWEB) 

In [ ]:
dataf1 = pd.read_csv('https://storage.googleapis.com/projetoaquecimente01/INSS_ori/CAT_INSS_01_2022.csv', sep=';', encoding = 'ISO-8859-1')

In [ ]:
dataf2 = pd.read_csv('https://storage.googleapis.com/projetoaquecimente01/INSS_ori/CAT_INSS_04_2022.csv', sep = ';',encoding = 'ISO-8859-1')

In [ ]:
dataf3 = pd.read_csv('https://storage.googleapis.com/projetoaquecimente01/INSS_ori/CAT_INSS_10_2022.csv', sep = ';',encoding = 'ISO-8859-1')

# 4 - Analisando os dados

 - Comunicação de Acidente de Trabalho - CAT

In [ ]:
#Verificando informações do DataFrame

dataf1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152140 entries, 0 to 152139
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   Agente  Causador  Acidente   152140 non-null  object
 1   Data Acidente                152140 non-null  object
 2   CBO                          152140 non-null  object
 3   CID-10                       152140 non-null  object
 4   CNAE2.0 Empregador           152140 non-null  int64 
 5   CNAE2.0 Empregador.1         152140 non-null  object
 6   Emitente CAT                 152140 non-null  object
 7   Espécie do benefício         152140 non-null  object
 8   Filiação Segurado            152140 non-null  object
 9   Indica Óbito Acidente        152140 non-null  object
 10  Munic Empr                   152140 non-null  object
 11  Natureza da Lesão            152140 non-null  object
 12  Origem de Cadastramento CAT  152140 non-null  object
 13  Parte Corpo At

In [ ]:
dataf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89602 entries, 0 to 89601
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Agente  Causador  Acidente   89602 non-null  object
 1   Data Acidente                89602 non-null  object
 2   CBO                          89602 non-null  object
 3   CID-10                       89602 non-null  object
 4   CNAE2.0 Empregador           89602 non-null  int64 
 5   CNAE2.0 Empregador.1         89602 non-null  object
 6   Emitente CAT                 89602 non-null  object
 7   Espécie do benefício         89602 non-null  object
 8   Filiação Segurado            89602 non-null  object
 9   Indica Óbito Acidente        89602 non-null  object
 10  Munic Empr                   89602 non-null  object
 11  Natureza da Lesão            89602 non-null  object
 12  Origem de Cadastramento CAT  89602 non-null  object
 13  Parte Corpo Atingida         89

In [ ]:
dataf3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15659 entries, 0 to 15658
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Agente  Causador  Acidente   15659 non-null  object
 1   Data Acidente                15659 non-null  object
 2   CBO                          15659 non-null  object
 3   CID-10                       15659 non-null  object
 4   CNAE2.0 Empregador           15659 non-null  int64 
 5   CNAE2.0 Empregador.1         15659 non-null  object
 6   Emitente CAT                 15659 non-null  object
 7   Espécie do benefício         15659 non-null  object
 8   Filiação Segurado            15659 non-null  object
 9   Indica Óbito Acidente        15659 non-null  object
 10  Munic Empr                   15659 non-null  object
 11  Natureza da Lesão            15659 non-null  object
 12  Origem de Cadastramento CAT  15659 non-null  object
 13  Parte Corpo Atingida         15

In [ ]:
dataf1.isna().sum()

Agente  Causador  Acidente     0
Data Acidente                  0
CBO                            0
CID-10                         0
CNAE2.0 Empregador             0
CNAE2.0 Empregador.1           0
Emitente CAT                   0
Espécie do benefício           0
Filiação Segurado              0
Indica Óbito Acidente          0
Munic Empr                     0
Natureza da Lesão              0
Origem de Cadastramento CAT    0
Parte Corpo Atingida           0
Sexo                           0
Tipo do Acidente               0
UF  Munic.  Acidente           0
UF Munic. Empregador           0
Data Acidente.1                0
Data Despacho Benefício        0
Data Acidente.2                0
Data Nascimento                0
Data Emissão CAT               0
CNPJ/CEI Empregador            0
dtype: int64

In [ ]:
dataf1

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,Munic Empr,Natureza da Lesão,Origem de Cadastramento CAT,Parte Corpo Atingida,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,CNPJ/CEI Empregador
0,{ñ class},2022/01,515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,Doenca Contagiosa ou Infecciosa (Tuberculos,Internet,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,02/08/1970,01/03/2022,57.571.275.002.570
1,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,Fratura,Internet,Braco (Entre O Punho a O Ombro),Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
2,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,Fratura,Internet,Braco (Entre O Punho a O Ombro),Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
3,"Aprision. Em, Sobre ou Entre Dois ou Mai",2022/02,514205-Coletor Lixo,S60.0 Contusao de Dedos s/Lesao da Unha,3811,Coleta de Residuos Nao-Perigosos,Empregador,Pa,Empregado,Não,520110-Anápolis,Luxacao,Internet,Punho,Masculino,Típico,{ñ class},Goiás,2022/02,0000/00,24/02/2022,19/09/1986,01/03/2022,00.000.000.000.000
4,Impacto de Pes. Contra Objeto em Movimento,2022/01,{ñ class},S90.0 Contusao do Tornozelo,4639,Comercio Atacadista de Produtos Alimenticios,Empregador,Pa,Empregado,Não,330100-Campos dos Goytacazes,Luxacao,Internet,Perna (Entre O Tornozelo e a Pelvis),Masculino,Típico,Maranhão,São Paulo,2022/01,0000/00,28/01/2022,25/07/1995,01/03/2022,00.000.000.000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152135,{ñ class},2022/04,000000-Não Informado,S82.6 Frat do Maleolo Lateral,0,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class,{ñ class},Masculino,Ignorado,Sergipe,Zerado,2022/04,2022/06,12/04/2022,08/11/1965,00/00/0000,00.000.000.000.000
152136,{ñ class},2022/04,000000-Não Informado,S82.1 Frat da Extremidade Proximal da Tibia,0,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class,{ñ class},Masculino,Ignorado,Sergipe,Zerado,2022/04,2022/06,30/04/2022,21/11/1985,00/00/0000,00.000.000.000.000
152137,{ñ class},2022/05,000000-Não Informado,S62.6 Frat de Outr Dedos,0,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class,{ñ class},Masculino,Ignorado,Sergipe,Zerado,2022/05,2022/06,05/05/2022,27/01/1992,00/00/0000,00.000.000.000.000
152138,{ñ class},2022/05,000000-Não Informado,S82.6 Frat do Maleolo Lateral,0,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class,{ñ class},Masculino,Ignorado,Sergipe,Zerado,2022/05,2022/06,15/05/2022,14/01/1975,00/00/0000,00.000.000.000.000


# 5 - Removendo algumas inconsistencias para upar no MongoDB

- Verificamos na tentativa de enviar para o MongoDB que o arquivo possuía alguns caracteres que não aceitava no Banco de Dados. Sendo assim, decidimos dar um replace no caraceteres para o banco de dados aceita e armazenar o Backup dos DF's

In [ ]:
dataf1.replace(['{', '}','\('],'',regex=True,inplace=True)

In [ ]:
dataf2.replace(['{', '}','\('],'',regex=True,inplace=True)

In [ ]:
dataf3.replace(['{', '}','\('],'',regex=True,inplace=True)

# 6 - Conexão ao MongoDB

## 6.1 - Configurando a conexão

In [ ]:
# Importando o MongoDB

import pymongo 
from pymongo import MongoClient

In [ ]:
# Realizando a conexão com o MongoDB

uri = "mongodb+srv://jhonathan-soulcode.akkbrfl.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='/content/X509-cert-9105300651307042956.pem')

In [ ]:
# Escolha do DataBase e Coleção

db = client['cat']
colecao1 = db['datasetorig1']
colecao2 = db['datasetorig2']
colecao3 = db['datasetorig3']

In [ ]:
# Verificar Conexão/Qtd. de documentos da coleção

colecao1.count_documents({})

152140

In [ ]:
  colecao2.count_documents({})

89602

In [ ]:
colecao3.count_documents({})

15659

## 6.2 - Exportando os arquivos originais para o MongoDB

In [ ]:
df_dict1 = dataf1.to_dict("records")

In [ ]:
# Alguns problemas com envio, resolvemos particionar o DF2 em 3 partes para upar no MongoDB

datataset2_part1 = dataf2[0:40001]
datataset2_part2 = dataf2[40001:80001]
datataset2_part3 = dataf2[80001:89603]

In [ ]:
df_dict2_part1 = datataset2_part1.to_dict("records")
df_dict2_part2 = datataset2_part2.to_dict("records")
df_dict2_part3 = datataset2_part3.to_dict("records")

In [ ]:
df_dict3 = dataf3.to_dict("records")

In [ ]:
# Enviando para o MONGO

# Os DFs tem inconsistencia e não está deixando upar no mongodb

colecao1.insert_many(df_dict1)

# DF2 estava com inconsistências diferentes do DF1

colecao2.insert_many(df_dict2_part1)
colecao2.insert_many(df_dict2_part2)
colecao2.insert_many(df_dict2_part3)

colecao3.insert_many(df_dict3)

# 7 - Unindo e analisando base de dados

## 7.1 -Concatenando os DF's

In [ ]:
# Primeriramente, decidimos juntas os 3 DataFrames e analisar

In [ ]:
#Concatenado os 3 data frames em um só que se chamará df
df = pd.concat([dataf1,dataf2,dataf3])

In [ ]:
# Linhas concatenadas
df

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,Munic Empr,Natureza da Lesão,Origem de Cadastramento CAT,Parte Corpo Atingida,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,CNPJ/CEI Empregador
0,ñ class,2022/01,515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,Doenca Contagiosa ou Infecciosa Tuberculos,Internet,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,02/08/1970,01/03/2022,57.571.275.002.570
1,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,Fratura,Internet,Braco Entre O Punho a O Ombro),Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
2,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,Fratura,Internet,Braco Entre O Punho a O Ombro),Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
3,"Aprision. Em, Sobre ou Entre Dois ou Mai",2022/02,514205-Coletor Lixo,S60.0 Contusao de Dedos s/Lesao da Unha,3811,Coleta de Residuos Nao-Perigosos,Empregador,Pa,Empregado,Não,520110-Anápolis,Luxacao,Internet,Punho,Masculino,Típico,ñ class,Goiás,2022/02,0000/00,24/02/2022,19/09/1986,01/03/2022,00.000.000.000.000
4,Impacto de Pes. Contra Objeto em Movimento,2022/01,ñ class,S90.0 Contusao do Tornozelo,4639,Comercio Atacadista de Produtos Alimenticios,Empregador,Pa,Empregado,Não,330100-Campos dos Goytacazes,Luxacao,Internet,Perna Entre O Tornozelo e a Pelvis),Masculino,Típico,Maranhão,São Paulo,2022/01,0000/00,28/01/2022,25/07/1995,01/03/2022,00.000.000.000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,Piso de Edificio - Superficie Utilizada Para,2022/09,ñ class,S62 Frat ao Nivel do Punho e da Mao,2330,"Fabricacao de Artefatos de Concreto, Cimento,",ñ class,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,410960-Guaratuba,Lesao Imediata,Internet,Mao Exceto Punho ou Dedos),Masculino,Típico,Roraima,Paraná,2022/09,2022/10,29/09/2022,18/09/1993,10/10/2022,21.161.901.000.135
15655,Escada Permanente Cujos Degraus Permitem Apoi,2022/10,232130-Prof. Física no Ensino Médio,S82 Frat da Perna Incl Tornozelo,8520,Ensino Medio,Autoridade Pública,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,410690-Curitiba,Fratura,Internet,"Perna Do Tornozelo, Exclusive, ao Joelho, Ex",Feminino,Típico,Roraima,Paraná,2022/10,2022/11,06/10/2022,24/11/1986,07/10/2022,76.416.965.000.121
15656,"Andaime, Plataforma - Edificio ou Estrutura",2022/09,517330-Vigilante,S82.7 Frat Mult da Perna,8112,Condominios Prediais,Segurado/Dependente,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,420460-Criciúma,Fratura,Internet,"Membros Inferiores, Partes Multiplas Qualque",Masculino,Típico,ñ class,Santa Catarina,2022/09,2022/10,20/09/2022,02/03/1986,10/10/2022,32.994.401.000.109
15657,"Martelo, Malho, Marreta- Ferramenta Manual Se",2022/10,724315-Soldador,S62.6 Frat de Outr Dedos,7112,Servicos de Engenharia,Empregador,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,411000-Iguaraçu,Fratura,Internet,Dedo,Masculino,Típico,Roraima,Paraná,2022/10,2022/10,05/10/2022,08/04/1984,06/10/2022,34.197.591.000.150


## 7.2 - Verificando o Index

In [ ]:
# O index está estranho, vamos modificar
df.loc[0,'Agente  Causador  Acidente']

0      ñ class                                    
0      ñ class                                    
0    Faca, Facao- Ferramenta Manual sem Forca Motr
Name: Agente  Causador  Acidente, dtype: object

In [ ]:
# Criamos um index

df.reset_index(inplace = True)

In [ ]:
# Setamos

df.set_index('index')

In [ ]:
df.loc[0,'Agente  Causador  Acidente']

'ñ class                                    '

In [ ]:
df.drop(['index'], axis = 1, inplace = True)

In [ ]:
# Inconsistência encontrada

df.loc[0,'Agente  Causador  Acidente']

'ñ class                                    '

# 8 - Buscando tirar as inconsistências brutas com o spark

## 8.1 - Configurando a Spark Session

In [ ]:
# Criando a SparkSession

spark = (
    SparkSession.builder
                .master('local')
                .appName('projaquecimento')
                .config('spark.ui.port', '4050')
                .getOrCreate()
)

In [ ]:
# Verificando se a mesma foi criada

spark

## 8.2 - Criando o esquema do Spark

In [ ]:
# Verificando os tipos de dados

df.dtypes

Agente  Causador  Acidente     object
Data Acidente                  object
CBO                            object
CID-10                         object
CNAE2.0 Empregador              int64
CNAE2.0 Empregador.1           object
Emitente CAT                   object
Espécie do benefício           object
Filiação Segurado              object
Indica Óbito Acidente          object
Munic Empr                     object
Natureza da Lesão              object
Origem de Cadastramento CAT    object
Parte Corpo Atingida           object
Sexo                           object
Tipo do Acidente               object
UF  Munic.  Acidente           object
UF Munic. Empregador           object
Data Acidente.1                object
Data Despacho Benefício        object
Data Acidente.2                object
Data Nascimento                object
Data Emissão CAT               object
CNPJ/CEI Empregador            object
dtype: object

In [ ]:
# Visualizando as colunas 

df.columns

Index(['Agente  Causador  Acidente', 'Data Acidente', 'CBO', 'CID-10',
       'CNAE2.0 Empregador', 'CNAE2.0 Empregador.1', 'Emitente CAT',
       'Espécie do benefício', 'Filiação Segurado', 'Indica Óbito Acidente',
       'Munic Empr', 'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador', 'Data Acidente.1',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

In [ ]:
# Primeiro esquema para importar os dados

esquema = (
    StructType([
        StructField('Agente  Causador  Acidente', StringType()),
        StructField('Data Acidente', StringType()),
        StructField('CBO', StringType()),
        StructField('CID-10', StringType()),
        StructField('CNAE2.0 Empregador', StringType()),
        StructField('CNAE2.0 Empregador.1', StringType()),
        StructField('Emitente CAT', StringType()),
        StructField('Espécie do benefício', StringType()),
        StructField('Filiação Segurado', StringType()),
        StructField('Indica Óbito Acidente', StringType()),
        StructField('Munic Empr', StringType()),
        StructField('Natureza da Lesão', StringType()),
        StructField('Origem de Cadastramento CAT', StringType()),
        StructField('Parte Corpo Atingida', StringType()),
        StructField('Sexo', StringType()),
        StructField('Tipo do Acidente', StringType()),
        StructField('UF  Munic.  Acidente', StringType()),
        StructField('UF Munic. Empregador', StringType()),
        StructField('Data Acidente.1', StringType()),
        StructField('Data Despacho Benefício', StringType()),
        StructField('Data Acidente.2', StringType()),
        StructField('Data Nascimento', StringType()),
        StructField('Data Emissão CAT', StringType()),
        StructField('CNPJ/CEI Empregador', StringType())
    ])
)

In [ ]:
# Criando o DataFrame Spark com o DF pandas

dfspark = spark.createDataFrame(df, schema = esquema)

In [ ]:
# Verificando o esquema

dfspark.printSchema()

root
 |-- Agente  Causador  Acidente: string (nullable = true)
 |-- Data Acidente: string (nullable = true)
 |-- CBO: string (nullable = true)
 |-- CID-10: string (nullable = true)
 |-- CNAE2.0 Empregador: string (nullable = true)
 |-- CNAE2.0 Empregador.1: string (nullable = true)
 |-- Emitente CAT: string (nullable = true)
 |-- Espécie do benefício: string (nullable = true)
 |-- Filiação Segurado: string (nullable = true)
 |-- Indica Óbito Acidente: string (nullable = true)
 |-- Munic Empr: string (nullable = true)
 |-- Natureza da Lesão: string (nullable = true)
 |-- Origem de Cadastramento CAT: string (nullable = true)
 |-- Parte Corpo Atingida: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo do Acidente: string (nullable = true)
 |-- UF  Munic.  Acidente: string (nullable = true)
 |-- UF Munic. Empregador: string (nullable = true)
 |-- Data Acidente.1: string (nullable = true)
 |-- Data Despacho Benefício: string (nullable = true)
 |-- Data Acidente.2: strin

In [ ]:
# Visualizando o DF
dfspark.show()

+--------------------------+-------------+--------------------+--------------------+------------------+--------------------+-------------------+--------------------+--------------------+---------------------+--------------------+--------------------+---------------------------+--------------------+-------------+----------------+--------------------+--------------------+---------------+-----------------------+---------------+---------------+----------------+-------------------+
|Agente  Causador  Acidente|Data Acidente|                 CBO|              CID-10|CNAE2.0 Empregador|CNAE2.0 Empregador.1|       Emitente CAT|Espécie do benefício|   Filiação Segurado|Indica Óbito Acidente|          Munic Empr|   Natureza da Lesão|Origem de Cadastramento CAT|Parte Corpo Atingida|         Sexo|Tipo do Acidente|UF  Munic.  Acidente|UF Munic. Empregador|Data Acidente.1|Data Despacho Benefício|Data Acidente.2|Data Nascimento|Data Emissão CAT|CNPJ/CEI Empregador|
+--------------------------+--------

In [ ]:
# Contabilizando as linhas do DF

dfspark.count()

257401

## 8.3 - Realizando um DropDuplicates

In [ ]:
# Verificando as linhas duplicadas com o spark (De acordo com o que foi passado, 
# o PySpark dropa as colunas que estão inteiramente repitidas com DropDuplicates)

# Esse algoritmo mostra a quantidade de linhas duplicadas

total_linhas = dfspark.count()
total_linhas_distintas = dfspark.distinct().count()

print(total_linhas - total_linhas_distintas)

100652


In [ ]:
# Dropando as colunas repitidas

dfspark1 = dfspark.dropDuplicates()

In [ ]:
dfspark1.count()

156749

In [ ]:
# Verificando a quantidade de colunas do Dataframe antigo

len(dfspark.columns)

24

In [ ]:
# Verificando a quantidade de colunas do Dataframe novo

len(dfspark1.columns)

24

# 9 - Tratando dados mais especifícos com o ***Pandas***

In [ ]:
# Convertendo Pandas para Spark

dfpandas = dfspark1.toPandas()

In [ ]:
dfpandas

## 9.1 - Backup 1.1

In [ ]:
# É sempre importante realizar um BACKUP antes de modificar os dados

dfback = dfpandas.copy()

##9.2 - Verificando e tratando os dados

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'CBO', 'CID-10', 'CNAE2.0 Empregador',
       'CNAE2.0 Empregador.1', 'Emitente CAT', 'Espécie do benefício',
       'Filiação Segurado', 'Indica Óbito Acidente', 'Munic Empr',
       'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

In [ ]:
# Verificando a quantiidade de valores distintos na colunas

dfspark1.select('Agente  Causador  Acidente').distinct().count()

294

- Verificando coluna: "Agente  Causador  Acidente"

In [ ]:
# Visualizando os valores

dfspark1.select('Agente  Causador  Acidente').distinct().show(295, truncate = False)

In [ ]:
# O ñ class foi a primeira inconsistencia encontrada

dfpandas['Agente  Causador  Acidente'].sort_values().unique()

In [ ]:
# Removendo a inconsistência para evitar colocar valor nulo na coluna Agente  Causador  Acidente


dfpandas['Agente  Causador  Acidente'].replace(['ñ class                                    '], np.NaN,inplace = True)
dfpandas.replace(['ñ class                                    '], np.NaN,inplace = True)

In [ ]:
dfpandas['Agente  Causador  Acidente'].sort_values().unique()

- Daqui em diante, iremos verificar as outras colunas

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'Data Acidente', 'CBO', 'CID-10',
       'CNAE2.0 Empregador', 'CNAE2.0 Empregador.1', 'Emitente CAT',
       'Espécie do benefício', 'Filiação Segurado', 'Indica Óbito Acidente',
       'Munic Empr', 'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador', 'Data Acidente.1',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

- Verificando coluna: "Data Acidente"

In [ ]:
dfpandas['Data Acidente'].sort_values().unique()

array(['2022/01', '2022/02', '2022/03', '2022/04', '2022/05', '2022/06',
       '2022/07', '2022/08', '2022/09', '2022/10'], dtype=object)

- Verificando coluna: "CBO"

In [ ]:
# Próxima coluna CBO

dfpandas['CBO'].sort_values().unique()

array(['000000-Não Informado                         ',
       '010105-Oficial General da Aeronáutica        ',
       '111120-Vereador                              ', ...,
       '992220-Pedreiro de Conservação de Vias Perman',
       '992225-Auxiliar Geral de Conservação de Vias ', 'nao_informado'],
      dtype=object)

In [ ]:
# Alterando o valor referente a coluna

dfpandas['CBO'].replace(['000000-Não Informado                         '], '000000-nao_informado                         ',inplace = True)
dfpandas['CBO'].replace(['nao_informado'], '000000-nao_informado                         ',inplace = True)

In [ ]:
dfpandas['CBO'].sort_values().unique()

array(['000000-nao_informado                         ',
       '010105-Oficial General da Aeronáutica        ',
       '111120-Vereador                              ', ...,
       '992215-Oper. de Ceifadeira na Conservação de ',
       '992220-Pedreiro de Conservação de Vias Perman',
       '992225-Auxiliar Geral de Conservação de Vias '], dtype=object)

In [ ]:
# Verificando a quantidade de valores

dfspark1.select('CBO').distinct().show(295, truncate = False)

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'Data Acidente', 'CBO', 'CID-10',
       'CNAE2.0 Empregador', 'CNAE2.0 Empregador.1', 'Emitente CAT',
       'Espécie do benefício', 'Filiação Segurado', 'Indica Óbito Acidente',
       'Munic Empr', 'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador', 'Data Acidente.1',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

- Verificando coluna: "CID-10"




In [ ]:
dfpandas['CID-10'].sort_values().unique()

array(['A00.0 Colera Dev Vibrio Cholerae 01 Biot Chol',
       'A01.0 Febre Tifoide                          ',
       'A02.0 Enterite p/Salmonela                   ', ...,
       'Z99.3 Dependencia de Cadeira de Rodas        ',
       'Z99.9 Depend Maquina e Aparelho Capacitante N', nan], dtype=object)

In [ ]:
dfpandas.replace(['ñ class                                    '], np.NaN,inplace = True)

In [ ]:
dfpandas['CID-10'].sort_values().unique()

array(['A00.0 Colera Dev Vibrio Cholerae 01 Biot Chol',
       'A01.0 Febre Tifoide                          ',
       'A02.0 Enterite p/Salmonela                   ', ...,
       'Z99.3 Dependencia de Cadeira de Rodas        ',
       'Z99.9 Depend Maquina e Aparelho Capacitante N', nan], dtype=object)

In [ ]:
# Dar um replace em todo o dataframe (Redundante)
dfpandas.replace(['ñ class                                    '], np.NaN ,inplace = True)

In [ ]:
dfpandas['CID-10'].sort_values().unique()

array(['A00.0 Colera Dev Vibrio Cholerae 01 Biot Chol',
       'A01.0 Febre Tifoide                          ',
       'A02.0 Enterite p/Salmonela                   ', ...,
       'Z99.3 Dependencia de Cadeira de Rodas        ',
       'Z99.9 Depend Maquina e Aparelho Capacitante N', nan], dtype=object)

- Verificando coluna: "CNAE2.0 Empregador"

In [ ]:
# Não foi encontrado nenhuma inconsistência visualmente
dfpandas['CNAE2.0 Empregador'].sort_values().unique()

- Verificando coluna: "CNAE2.0 Empregador.1"

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'Data Acidente', 'CBO', 'CID-10',
       'CNAE2.0 Empregador', 'CNAE2.0 Empregador.1', 'Emitente CAT',
       'Espécie do benefício', 'Filiação Segurado', 'Indica Óbito Acidente',
       'Munic Empr', 'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador', 'Data Acidente.1',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

In [ ]:
# Não foi encontrado nenhuma inconsistência visualmente

dfpandas['CNAE2.0 Empregador.1'].sort_values().unique()

- Verificando coluna: "Emitente CAT"

In [ ]:
dfpandas['Emitente CAT'].sort_values().unique()

array(['Autoridade Pública ', 'Empregador         ',
       'Médico             ', 'Segurado/Dependente',
       'Sindicato          ', 'ñ class          '], dtype=object)

In [ ]:
dfpandas.replace(['ñ class          '], np.NaN ,inplace = True)

In [ ]:
dfpandas['Emitente CAT'].sort_values().unique()

array(['Autoridade Pública ', 'Empregador         ',
       'Médico             ', 'Segurado/Dependente',
       'Sindicato          ', nan], dtype=object)

- Verificando coluna: "Espécie do benefício"

In [ ]:
# Não foi encontrado nenhuma inconsistência visualmente

dfpandas['Espécie do benefício'].sort_values().unique()

array(['Aposent. Invalidez Acidente Trabalho         ',
       'Auxílio Acidente                             ',
       'Auxílio Doenca por Acidente do Trabalho      ',
       'Pa                                           ',
       'Pensão por Morte Acidente do Trabalho        '], dtype=object)

- Verificando coluna: "Filiação Segurado"

In [ ]:
# Não foi encontrado nenhuma inconsistência visualmente

dfpandas['Filiação Segurado'].sort_values().unique()

array(['Empregado                                    ',
       'Segurado Especial                            ',
       'Trabalhador Avulso                           ', nan], dtype=object)

- Verificando coluna: "Indica Óbito Acidente"

In [ ]:
dfpandas['Indica Óbito Acidente'].sort_values().unique()

array(['Não', 'Sim', 'ñ '], dtype=object)

In [ ]:
dfpandas.replace(['ñ '], np.NaN ,inplace = True)

- Verificando coluna: "Munic Empr"

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'Data Acidente', 'CBO', 'CID-10',
       'CNAE2.0 Empregador', 'CNAE2.0 Empregador.1', 'Emitente CAT',
       'Espécie do benefício', 'Filiação Segurado', 'Indica Óbito Acidente',
       'Munic Empr', 'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador', 'Data Acidente.1',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

In [ ]:
dfpandas['Munic Empr'].sort_values().unique()

array(['000000-Ignorado                          ',
       '110002-Ariquemes                         ',
       '110003-Cabixi                            ', ...,
       '522230-Vila Propício                     ',
       '530010-Brasília                          ', nan], dtype=object)

In [ ]:
dfpandas.replace(['ñ Class                                '], np.NaN ,inplace = True)

In [ ]:
dfpandas['Munic Empr'].sort_values().unique()

array(['000000-Ignorado                          ',
       '110002-Ariquemes                         ',
       '110003-Cabixi                            ', ...,
       '522230-Vila Propício                     ',
       '530010-Brasília                          ', nan], dtype=object)

- Verificando coluna: "Natureza da Lesão"




In [ ]:
# Não foi encontrado nenhuma inconsistência visualmente

dfpandas['Natureza da Lesão'].sort_values().unique()

array(['Amputacao ou Enucleacao                      ',
       'Asfixia, Estrangulamento, Afogamento         ',
       'Choque Eletrico e Eletroplessao Eletrocuss  ',
       'Concussao Cerebral                           ',
       'Congelamento, Geladura e Outros Efeitos Da   ',
       'Contusao, Esmagamento Superficie Cutanea I  ',
       'Corte, Laceracao, Ferida Contusa, Punctura   ',
       'Dermatose Erupcao, Inflamacao da Pele, Inc  ',
       'Distensao, Torcao                            ',
       'Doenca Contagiosa ou Infecciosa Tuberculos  ',
       'Doenca, Nic                                  ',
       'Efeito de Radiacao Imediato) - Queimadura   ',
       'Efeito de Radiacao Mediato) - Queimadura D  ',
       'Envenenamento Sistemico - Condicao Morbida   ',
       'Escoriacao, Abrasao Ferimento Superficial)  ',
       'Fratura                                      ',
       'Hernia de Qualquer Natureza, Ruptura         ',
       'Inflamacao de Articulacao, Tendao ou Muscul  ',

- Verificando coluna: "Origem de Cadastramento CAT"

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'Data Acidente', 'CBO', 'CID-10',
       'CNAE2.0 Empregador', 'CNAE2.0 Empregador.1', 'Emitente CAT',
       'Espécie do benefício', 'Filiação Segurado', 'Indica Óbito Acidente',
       'Munic Empr', 'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador', 'Data Acidente.1',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

In [ ]:
dfpandas['Origem de Cadastramento CAT'].sort_values().unique()

array(['Internet', 'ñ class'], dtype=object)

In [ ]:
dfpandas.replace(['ñ class'], np.NaN ,inplace = True)

- Verificando coluna: "Parte Corpo Atingida"

In [ ]:
dfpandas['Parte Corpo Atingida'].sort_values().unique()

array(['Abdome Inclusive Orgaos Internos)           ',
       'Antebraco Entre O Punho e O Cotovelo)       ',
       'Aparelho Circulatorio                        ',
       'Aparelho Digestivo                           ',
       'Aparelho Genito-Urinario                     ',
       'Aparelho Respiratorio                        ',
       'Artelho                                      ',
       'Articulacao do Tornozelo                     ',
       'Boca Inclusive Labios, Dentes, Lingua, Garga',
       'Braco Acima do Cotovelo)                    ',
       'Braco Entre O Punho a O Ombro)              ',
       'Cabeca, Nic                                  ',
       'Cabeca, Partes Multiplas Qualquer Combinacao',
       'Cotovelo                                     ',
       'Coxa                                         ',
       'Cranio Inclusive Encefalo)                  ',
       'Dedo                                         ',
       'Dorso Inclusive Musculos Dorsais, Coluna e M',


- Verificando coluna: "Sexo"

In [ ]:
# Não foi encontrado nenhuma inconsistência visualmente

dfpandas['Sexo'].sort_values().unique()

array(['Feminino     ', 'Indeterminado', 'Masculino    ', 'Não Informado'],
      dtype=object)

- Verificando coluna: "Tipo do Acidente"

In [ ]:
dfpandas['Tipo do Acidente'].sort_values().unique()

array(['Doença  ', 'Ignorado', 'Trajeto ', 'Típico  '], dtype=object)

- Verificando coluna: "UF  Munic.  Acidente"

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'Data Acidente', 'CBO', 'CID-10',
       'CNAE2.0 Empregador', 'CNAE2.0 Empregador.1', 'Emitente CAT',
       'Espécie do benefício', 'Filiação Segurado', 'Indica Óbito Acidente',
       'Munic Empr', 'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador', 'Data Acidente.1',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

In [ ]:
dfpandas['UF  Munic.  Acidente'].sort_values().unique()

array(['Acre            ', 'Alagoas         ', 'Amapá           ',
       'Amazonas        ', 'Ceará           ', 'Maranhão        ',
       'Paraíba         ', 'Pará            ', 'Pernambuco      ',
       'Piauí           ', 'Rio Grande Norte', 'Rondônia        ',
       'Roraima         ', 'Sergipe         ', 'Tocantins       ',
       'Zerado          ', 'ñ class       '], dtype=object)

In [ ]:
dfpandas.replace(['ñ class       '], np.NaN,inplace = True)

In [ ]:
dfpandas['UF  Munic.  Acidente'].sort_values().unique()

array(['Acre            ', 'Alagoas         ', 'Amapá           ',
       'Amazonas        ', 'Ceará           ', 'Maranhão        ',
       'Paraíba         ', 'Pará            ', 'Pernambuco      ',
       'Piauí           ', 'Rio Grande Norte', 'Rondônia        ',
       'Roraima         ', 'Sergipe         ', 'Tocantins       ',
       'Zerado          ', nan], dtype=object)

In [ ]:
dfpandas['UF  Munic.  Acidente'].sort_values().unique()

array(['Acre            ', 'Alagoas         ', 'Amapá           ',
       'Amazonas        ', 'Ceará           ', 'Maranhão        ',
       'Paraíba         ', 'Pará            ', 'Pernambuco      ',
       'Piauí           ', 'Rio Grande Norte', 'Rondônia        ',
       'Roraima         ', 'Sergipe         ', 'Tocantins       ',
       'Zerado          ', nan], dtype=object)

- Verificando coluna: "UF Munic. Empregador"

In [ ]:
dfpandas['UF Munic. Empregador'].sort_values().unique()

array(['Acre               ', 'Alagoas            ',
       'Amapá              ', 'Amazonas           ',
       'Bahia              ', 'Ceará              ',
       'Distrito Federal   ', 'Espírito Santo     ',
       'Goiás              ', 'Maranhão           ',
       'Mato Grosso        ', 'Mato Grosso do Sul ',
       'Minas Gerais       ', 'Paraná             ',
       'Paraíba            ', 'Pará               ',
       'Pernambuco         ', 'Piauí              ',
       'Rio Grande do Norte', 'Rio Grande do Sul  ',
       'Rio de Janeiro     ', 'Rondônia           ',
       'Roraima            ', 'Santa Catarina     ',
       'Sergipe            ', 'São Paulo          ',
       'Tocantins          ', 'Zerado             '], dtype=object)

In [ ]:
dfpandas.replace(['Zerado             '], np.NaN,inplace = True)

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'Data Acidente', 'CBO', 'CID-10',
       'CNAE2.0 Empregador', 'CNAE2.0 Empregador.1', 'Emitente CAT',
       'Espécie do benefício', 'Filiação Segurado', 'Indica Óbito Acidente',
       'Munic Empr', 'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador', 'Data Acidente.1',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

- Verificando as Datas

In [ ]:
dfpandas['Data Acidente.1'].sort_values().unique()

array(['2022/01', '2022/02', '2022/03', '2022/04', '2022/05', '2022/06',
       '2022/07', '2022/08', '2022/09', '2022/10'], dtype=object)

In [ ]:
dfpandas['Data Acidente.2'].sort_values().unique()

In [ ]:
dfpandas['Data Acidente'].sort_values().unique()

array(['2022/01', '2022/02', '2022/03', '2022/04', '2022/05', '2022/06',
       '2022/07', '2022/08', '2022/09', '2022/10'], dtype=object)

In [ ]:
dfpandas['Data Emissão CAT'].sort_values().unique()

array(['00/00/0000', '01/03/2022', '01/06/2022', '01/10/2022',
       '02/03/2022', '02/06/2022', '02/10/2022', '03/03/2022',
       '03/06/2022', '03/10/2022', '04/03/2022', '04/06/2022',
       '04/10/2022', '05/03/2022', '05/06/2022', '05/10/2022',
       '06/03/2022', '06/06/2022', '06/10/2022', '07/03/2022',
       '07/06/2022', '07/10/2022', '08/03/2022', '08/06/2022',
       '08/10/2022', '09/03/2022', '09/06/2022', '09/10/2022',
       '10/03/2022', '10/06/2022', '10/10/2022', '11/03/2022',
       '11/06/2022', '11/10/2022', '12/03/2022', '12/06/2022',
       '12/10/2022', '13/03/2022', '13/06/2022', '13/10/2022',
       '14/03/2022', '14/06/2022', '14/10/2022', '15/03/2022',
       '15/06/2022', '15/10/2022', '16/03/2022', '16/06/2022',
       '16/10/2022', '17/03/2022', '17/06/2022', '17/10/2022',
       '18/03/2022', '18/06/2022', '18/10/2022', '19/03/2022',
       '19/06/2022', '19/10/2022', '20/03/2022', '20/06/2022',
       '20/10/2022', '21/03/2022', '21/06/2022', '21/10

In [ ]:
# Dropando dados inconsistentes

dfpandas.replace(['00/00/0000'], np.NaN,inplace = True)

In [ ]:
dfpandas['Data Despacho Benefício'].sort_values().unique()

array(['0000/00', '2022/01', '2022/02', '2022/03', '2022/04', '2022/05',
       '2022/06', '2022/07', '2022/10', '2022/11'], dtype=object)

In [ ]:
dfpandas.replace(['0000/00'], np.NaN,inplace = True)

In [ ]:
dfpandas.columns

Index(['Agente  Causador  Acidente', 'CBO', 'CID-10', 'CNAE2.0 Empregador',
       'CNAE2.0 Empregador.1', 'Emitente CAT', 'Espécie do benefício',
       'Filiação Segurado', 'Indica Óbito Acidente', 'Munic Empr',
       'Natureza da Lesão', 'Origem de Cadastramento CAT',
       'Parte Corpo Atingida', 'Sexo', 'Tipo do Acidente',
       'UF  Munic.  Acidente', 'UF Munic. Empregador',
       'Data Despacho Benefício', 'Data Acidente.2', 'Data Nascimento',
       'Data Emissão CAT', 'CNPJ/CEI Empregador'],
      dtype='object')

In [ ]:
dfpandas['CNPJ/CEI Empregador']

0         00.000.000.000.000
1         00.000.000.000.000
2         00.000.000.000.000
3         00.000.000.000.000
4         18.715.508.000.131
                 ...        
156744    00.000.000.000.000
156745    00.000.000.000.000
156746    00.000.000.000.000
156747    00.000.000.000.000
156748    00.000.000.000.000
Name: CNPJ/CEI Empregador, Length: 156749, dtype: object

In [ ]:
dfpandas.loc[156747,'CNPJ/CEI Empregador']

'00.000.000.000.000'

In [ ]:
dfpandas['CNPJ/CEI Empregador'].sort_values().unique()

array(['00.000.000.000.000', '00.000.000.000.191', '00.000.000.014.737',
       ..., '98.110.000.000.149', '98.416.225.000.128',
       '98.714.876.000.102'], dtype=object)

In [ ]:
# Dropando dados inconsistentes

dfpandas.replace('00.000.000.000.000',np.NaN,regex=True,inplace=True)

In [ ]:
dfpandas.groupby(['emitente_cat']).size().sort_values(ascending=False)

In [ ]:
dfpandas.groupby(['Indica Óbito Acidente']).size().sort_values(ascending=False)

Indica Óbito Acidente
Não    116191
Sim       603
dtype: int64

## 9.3 - Dropando colunas

In [ ]:
ver = dfpandas['CNAE2.0 Empregador.1'] == dfpandas['CNAE2.0 Empregador']

ver.unique()

array([False])

In [ ]:
# As colunas possuem dados iguais e irrelevante no que diz respeito a data, pois Data Acidente.2 possuem datas completas

dfpandas.drop(['Data Acidente.1','Data Acidente'],axis=1,inplace=True)


# 10 - Renomeando colunas com PySpark

In [ ]:
dfpandas.dtypes

Agente  Causador  Acidente     object
CBO                            object
CID-10                         object
CNAE2.0 Empregador             object
CNAE2.0 Empregador.1           object
Emitente CAT                   object
Espécie do benefício           object
Filiação Segurado              object
Indica Óbito Acidente          object
Munic Empr                     object
Natureza da Lesão              object
Origem de Cadastramento CAT    object
Parte Corpo Atingida           object
Sexo                           object
Tipo do Acidente               object
UF  Munic.  Acidente           object
UF Munic. Empregador           object
Data Despacho Benefício        object
Data Acidente.2                object
Data Nascimento                object
Data Emissão CAT               object
CNPJ/CEI Empregador            object
dtype: object

## 10.1 - Criando o segundo esquema

In [ ]:
esquema2 = (
    StructType([
        StructField('Agente  Causador  Acidente', StringType()),
        StructField('CBO', StringType()),
        StructField('CID-10', StringType()),
        StructField('CNAE2.0 Empregador', StringType()),
        StructField('CNAE2.0 Empregador.1', StringType()),
        StructField('Emitente CAT', StringType()),
        StructField('Espécie do benefício', StringType()),
        StructField('Filiação Segurado', StringType()),
        StructField('Indica Óbito Acidente', StringType()),
        StructField('Munic Empr', StringType()),
        StructField('Natureza da Lesão', StringType()),
        StructField('Origem de Cadastramento CAT', StringType()),
        StructField('Parte Corpo Atingida', StringType()),
        StructField('Sexo', StringType()),
        StructField('Tipo do Acidente', StringType()),
        StructField('UF  Munic.  Acidente', StringType()),
        StructField('UF Munic. Empregador', StringType()),
        StructField('Data Despacho Benefício', StringType()),
        StructField('Data Acidente.2', StringType()),
        StructField('Data Nascimento', StringType()),
        StructField('Data Emissão CAT', StringType()),
        StructField('CNPJ/CEI Empregador', StringType())
    ])
)

In [ ]:
# Convertendo Pandas para PySpark

dfspark2 = spark.createDataFrame(dfpandas, schema = esquema2)

In [ ]:
dfspark2.show()

## 10.2 - Renomeando as colunas

In [ ]:
# Renomeando as colunas para um padrão melhor

dfspark3 = (dfspark2.withColumnRenamed('Agente  Causador  Acidente', 'agente_causador_acidente')
                     .withColumnRenamed('CBO', 'cbo')
                     .withColumnRenamed('CID-10', 'cid_10')
                     .withColumnRenamed('CNAE2.0 Empregador', 'cnae20')
                     .withColumnRenamed('CNAE2.0 Empregador.1', 'cnae20_1')
                     .withColumnRenamed('Emitente CAT', 'emitente_cat')
                     .withColumnRenamed('Espécie do benefício', 'especie_beneficio')
                     .withColumnRenamed('Filiação Segurado', 'filiacao_segurado')
                     .withColumnRenamed('Indica Óbito Acidente', 'indica_obito_acidente')
                     .withColumnRenamed('Munic Empr', 'munic_empr')
                     .withColumnRenamed('Natureza da Lesão', 'natureza_lesao')
                     .withColumnRenamed('Origem de Cadastramento CAT', 'orig_cadastramento_cat')
                     .withColumnRenamed('Parte Corpo Atingida', 'parte_corpo_atingida')
                     .withColumnRenamed('Sexo', 'sexo')
                     .withColumnRenamed('Tipo do Acidente', 'derivacao_acidente')
                     .withColumnRenamed('UF  Munic.  Acidente', 'uf_mun_acidente')
                     .withColumnRenamed('UF Munic. Empregador', 'uf_munic_empregador')
                     .withColumnRenamed('Data Acidente.2', 'data_acidente')
                     .withColumnRenamed('Data Nascimento', 'data_nascimento')
                     .withColumnRenamed('Data Emissão CAT', 'data_emissao_cat')
                     .withColumnRenamed('CNPJ/CEI Empregador', 'cnpj_cei_empregado')

)

# 11 - Criando 2 novas colunas

In [ ]:
#Criando a coluna CNAE_Empregador juntando os dados de CNAE2.0 Empregador' e 'CNAE2.0 Empregador.1'

dfspark4 = dfspark3.withColumn('cna_empregador', F.concat(F.col('cnae20'),F.lit(' - '), F.col('cnae20_1')))
dfspark4.show(truncate=False)

+---------------------------------------------+---------------------------------------------+---------------------------------------------+------+---------------------------------------------+-------------------+---------------------------------------------+---------------------------------------------+---------------------+-----------------------------------------+---------------------------------------------+----------------------+---------------------------------------------+-------------+------------------+----------------+-------------------+-----------------------+-------------+---------------+----------------+------------------+----------------------------------------------------+
|agente_causador_acidente                     |cbo                                          |cid_10                                       |cnae20|cnae20_1                                     |emitente_cat       |especie_beneficio                            |filiacao_segurado                            

In [ ]:
#Agora precisamos dropar as duas colunas 'cnae20' e 'cnae20_1' que foram juntada, agora existe uma com os valores agrupados
dfspark4 = dfspark4.drop(F.col('cnae20'))
dfspark4 = dfspark4.drop(F.col('cnae20_1'))

In [ ]:
dfspark4.show()

In [ ]:
dfspark5 = dfspark4.withColumn('tipo_lesao', F.concat(F.col('natureza_lesao'),F.lit('-'), F.col('parte_corpo_atingida')))
dfspark5.show(truncate=False)

+---------------------------------------------+---------------------------------------------+---------------------------------------------+-------------------+---------------------------------------------+---------------------------------------------+---------------------+-----------------------------------------+---------------------------------------------+----------------------+---------------------------------------------+-------------+------------------+----------------+-------------------+-----------------------+-------------+---------------+----------------+------------------+----------------------------------------------------+-------------------------------------------------------------------------------------------+
|agente_causador_acidente                     |cbo                                          |cid_10                                       |emitente_cat       |especie_beneficio                            |filiacao_segurado                            |indica_obito_

In [ ]:
#CONVERTER UMA DATA DO FORMATO STRING PARA O FORMATO DATE - yyyy-MM-dd

# O MongoDB teve problema com esse formato

dfspark6 = dfspark5.withColumn('data_acidente', F.to_date(F.col('data_acidente'), 'dd/MM/yyyy'))

In [ ]:
dfspark7 = dfspark6.withColumn('data_nascimento', F.to_date(F.col('data_nascimento'), 'dd/MM/yyyy'))

In [ ]:
dfspark8 = dfspark7.withColumn('data_emissao_cat', F.to_date(F.col('data_emissao_cat'), 'dd/MM/yyyy'))

In [ ]:
dfspark5.printSchema()

# 12 - Filtro e GROUPBY

In [ ]:
dfpandas2 = dfspark5.toPandas()

In [ ]:
dfpandas2['sexo'].unique()

array(['Masculino    ', 'Feminino     ', 'Não Informado', 'Indeterminado'],
      dtype=object)

In [ ]:
# Filtro de de sexo feminino
ftsexof = dfpandas2['sexo'] == 'Feminino     '
dfpandas2.loc[ftsexof]


In [ ]:
# Filtro de de sexo masculino
ftsexof = dfpandas2['sexo'] == 'Masculino    '
dfpandas2.loc[ftsexof]


In [ ]:
dfpandas2.groupby(['sexo']).size().sort_values(ascending=False)

sexo
Masculino        102917
Feminino          53328
Não Informado       502
Indeterminado         2
dtype: int64

In [ ]:
#Agrupando 'Natureza da Lesão' e exibindo do maior para o menor
dfpandas2.groupby(['natureza_lesao']).size().sort_values(ascending=False)

natureza_lesao
NaN                                              39955
Fratura                                          21578
Corte, Laceracao, Ferida Contusa, Punctura       20372
Lesao Imediata                                   12712
Contusao, Esmagamento Superficie Cutanea I       11909
Escoriacao, Abrasao Ferimento Superficial)       10409
Distensao, Torcao                                 8539
Luxacao                                           6745
Outras Lesoes, Nic                                6604
Doenca Contagiosa ou Infecciosa Tuberculos        3311
Lesao Imediata, Nic                               3182
Queimadura ou Escaldadura - Efeito de Tempe       2885
Lesoes Multiplas                                  2110
Doenca, Nic                                       1983
Inflamacao de Articulacao, Tendao ou Muscul        998
Amputacao ou Enucleacao                            898
Queimadura Quimica Lesao de Tecido Provoca         773
Concussao Cerebral                                

# 13 - Window Function

In [ ]:
# Importando as window function

from pyspark.sql.window import Window

In [ ]:
w0 = Window.partitionBy(F.col('uf_mun_acidente')).orderBy(F.col('sexo').desc())

In [ ]:
dfspark10 = dfspark5.withColumn('numero_linha', F.row_number().over(w0))

In [ ]:
dfspark10.show()

+------------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------------------+--------------------+--------------------+----------------------+--------------------+-------------+------------------+----------------+-------------------+-----------------------+-------------+---------------+----------------+------------------+--------------------+--------------------+------------+
|agente_causador_acidente|                 cbo|              cid_10|       emitente_cat|   especie_beneficio|   filiacao_segurado|indica_obito_acidente|          munic_empr|      natureza_lesao|orig_cadastramento_cat|parte_corpo_atingida|         sexo|derivacao_acidente| uf_mun_acidente|uf_munic_empregador|Data Despacho Benefício|data_acidente|data_nascimento|data_emissao_cat|cnpj_cei_empregado|      cna_empregador|          tipo_lesao|numero_linha|
+------------------------+--------------------+--------------------+-------------------+----

# 14 - Enviando o dataframe tratado para GCP

## 14.1 - Salvado o arquivo em csv

In [ ]:
dfpandas2.to_csv('cat_tratado.csv')

## 14.2 - Enviando para GCP

In [ ]:
client = storage.Client()

bucket = client.get_bucket('projetoaquecimente01')
blob = bucket.blob('INSS_trat/CAT_tratado')
blob.upload_from_filename('/content/cat_tratado.csv')

## 14.3 - Enviando para o MongoDB

In [ ]:
dfpandas3 = dfspark5.toPandas()

In [ ]:
dfpandas3

In [ ]:
#Escolha do DataBase e Coleção
db = client['cat_tratado']
colecao_trat1 = db['dataset_trat']

# Realizando o Save e a inserção

datataset5_part1 = dfpandas2[0:40001]
datataset5_part2 = dfpandas2[40001:80001]
datataset5_part3 = dfpandas2[80001:100001]
datataset5_part4 = dfpandas2[100001:120001]
datataset5_part5 = dfpandas2[120001:156750]

df_dict5_part1 = datataset5_part1.to_dict("records")
df_dict5_part2 = datataset5_part2.to_dict("records")
df_dict5_part3 = datataset5_part3.to_dict("records")
df_dict5_part4 = datataset5_part4.to_dict("records")
df_dict5_part5 = datataset5_part5.to_dict("records")

colecao_trat1.insert_many(df_dict5_part1)
colecao_trat1.insert_many(df_dict5_part2)
colecao_trat1.insert_many(df_dict5_part3)
colecao_trat1.insert_many(df_dict5_part4)
colecao_trat1.insert_many(df_dict5_part5)

# ! Pontos para analisar posteriormente !

- Falta upar para o MySQL
- Remover espaços sobrando e algumas inconsistências nos dados
- Fazer os Plots com Pandas e Insights SparkSQL

